In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from ast import literal_eval
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_excel("check.xlsx")
df.head()

,case number,Date of the judgement,judge one,judge two,judge3 three,Appellant,Respondent,appeal from,judge one conclusion,judge two conclusion,judge three conclusion,length of judgement,full judgement,year,month,day
0,C1/2011/0694,2012-01-12,LORD JUSTICE WARD,none,none,The Queen on the application of JBol Limited,The Health Protection Agency,QUEEN'S BENCH DIVISIONADMINISTRATIVE COURTMR ...,I appreciate how important this case is for th...,none,none,20110,Lord Justice Ward:This is a renewed applicatio...,2012,Jan,Thu
1,B3/2010/2462,2012-01-20,LORD JUSTICE GROSS,MR JUSTICE MANN,SIR STEPHEN SEDLEY,Charnock & Ors,Rowan & Ors,HHJ Gore QC7LV18468 / 8LV12546,It may be said that this reverses the notice r...,I agree.,I also agree.,22963,Sir Stephen Sedley:On 30 June 2007 in Ullet Ro...,2012,Jan,Fri
2,B2/2011/0663,2012-01-18,LORD JUSTICE GROSS,MR JUSTICE MANN,SIR STEPHEN SEDLEY,GARY TAYLOR,(1) TERENCE LAMBERT\n(2) MICHELLE LAMBERT,THE DONCASTER COUNTY COURTHIS HONOUR JUDGE MOORE,It may be said that this reverses the notice r...,I agree.,I also agree.,42643,Lord Justice Lloyd:IntroductionThis is an appe...,2012,Jan,Wed
3,A3/2011/0526,2012-01-18,LORD JUSTICE MUMMERY,LORD JUSTICE JACKSON,LORD JUSTICE LEWISON,DANIEL THOMAS BRIAN HORLER,DAVID RUBIN & ORS,THE CHANCERY DIVISIONHis Honour Judge Raynor ...,As rules 8.1 (6) and 8.3 (6) make clear it is ...,I agree.,I also agree.,18105,Lord Justice Lewison:Mr Andrew Millar was adju...,2012,Jan,Wed
4,A3/2011/0402,2012-01-20,LORD JUSTICE TOULSON,LORD JUSTICE ETHERTON,LORD JUSTICE KITCHIN,Apimed Medical Honey Limited\n(a company incor...,Brightwake Limited\n(trading as Advancis Medical),THE PATENTS COUNTY COURTHis Honour Judge Fysh...,The Comptroller served a respondent's notice b...,I agree,I also agree,40573,Lord Justice Kitchin:IntroductionThis is a dis...,2012,Jan,Fri


In [3]:
df.tail()

,case number,Date of the judgement,judge one,judge two,judge3 three,Appellant,Respondent,appeal from,judge one conclusion,judge two conclusion,judge three conclusion,length of judgement,full judgement,year,month,day
2253,CA-2021-00065,2022-05-16,LORD JUSTICE LEWISON,LADY JUSTICE ASPLIN,LORD JUSTICE EDIS,(1) THE SECRETARY OF STATE FOR TRANSPORT\n(2) ...,ELLIOTT CUCIUREAN,THE HIGH COURT OF JUSTICECHANCERY DIVISIONBIR...,"""…requires an appellate court to exercise a de...",I agree.,I also agree.,54014,This judgment was handed down remotely by circ...,2022,May,Mon
2254,CA-2021-00072,2022-05-17,LORD JUSTICE DINGEMANS,LADY JUSTICE ANDREWS,LADY JUSTICE ELISABETH LAING,THE QUEEN (on the application of [KALONGA]),LONDON BOROUGH OF CROYDON,THE HIGH COURT OF JUSTICEQUEEN'S BENCH DIVISI...,The review of its decision which the local aut...,"For those reasons, which accord with those exp...",I agree with both judgments.,66131,This judgment was handed down remotely by circ...,2022,May,Tue
2255,CA-2021-00328,2022-05-17,LORD JUSTICE COULSON,LORD JUSTICE ARNOLD,LORD JUSTICE PHILLIPS,(1) Mrs Iman Said Abdul Al-Rawas\n(2) Mr Thame...,(1) Hassan Khan & Co (A Firm)\n(2) The Khan Pa...,THE QUEENS BENCH DIVISIONMr Justice Morris[20...,"For all these reasons, therefore, I consider t...",I agree.,I also agree.,19604,This judgment will be handed down remotely by ...,2022,May,Tue
2256,CA-2021-00141,2022-05-19,LORD JUSTICE PETER JACKSON,LADY JUSTICE SIMLER,LADY JUSTICE ANDREWS,YZ,THE CHIEF CONSTABLE OF SOUTH WALES POLICE,THE HIGH COURT OF JUSTICEQUEEN'S BENCH DIVISI...,GROUND 4 Ground 4 adds nothing to the other gr...,I agree.,I also agree.,51074,This judgment was handed down by the Judge rem...,2022,May,Thu
2257,CA-2022-00041,2022-05-27,LORD JUSTICE NEWEY,LORD JUSTICE ARNOLD,LORD JUSTICE BIRSS,(1) NEURIM PHARMACEUTICALS (1991) LIMITED\n(2)...,(1) GENERICS (UK) LIMITED\n(2) VIATRIS UK HEAL...,"THE HIGHT COURT OF JUSTICE,BUSINESS AND PROPE...","Fourthly, the judge was entitled to reject Pro...",I agree.,I also agree.,44488,This judgment was handed down by the Court rem...,2022,May,Fri


In [4]:
# a function to perform text preprocessing
def remove_stop_words(text):
    #text= " ".join([w.lower() for w in literal_eval(text)] )    
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

df["clean"]=df["judge one conclusion"].apply(remove_stop_words)

In [5]:
# seperating text data with dismiss appeal or allow appeal from text data without the keyword
a=[]
for i in df["clean"]:
    if i.find("dismiss appeal") !=-1:
        a.append("dismiss appeal")
    elif i.find("allow appeal") !=-1:
        a.append("allow appeal")
    else:
        a.append("no")
df["case_status"]=  a

labe_data=df.query("case_status=='dismiss appeal' | case_status=='allow appeal'")["judge one conclusion"]
unlabe_data=df.query("case_status=='no'")["judge one conclusion"]

In [6]:
# Convert labeled and unlabeled text data into numerical features using TfidfVectorizer
vectorizer = TfidfVectorizer()
X_labeled = vectorizer.fit_transform(labe_data)
X_unlabeled = vectorizer.transform(unlabe_data)


# Hyperparameter Tuning: Find the best number of clusters using silhouette score
best_score = -1
best_num_clusters = None
for num_clusters in range(2, 6):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(X_labeled)
    cluster_labels = kmeans.labels_
    silhouette_avg = silhouette_score(X_labeled, cluster_labels)
    if silhouette_avg > best_score:
        best_score = silhouette_avg
        best_num_clusters = num_clusters

print("Best Number of Clusters:", best_num_clusters)

# Initialize KMeans clustering algorithm with the best number of clusters
kmeans = KMeans(n_clusters=best_num_clusters, random_state=42)

# Dimensionality Reduction using PCA for labeled data
pca = PCA(n_components=2)
X_pca_labeled = pca.fit_transform(X_labeled.toarray())

# Fit the KMeans model on the labeled data
kmeans.fit(X_pca_labeled)

# Get the cluster assignments for each labeled data point
labeled_df = kmeans.labels_

# Apply KMeans clustering to the unlabeled data using the same model
X_pca_unlabeled = pca.transform(X_unlabeled.toarray())
#unlabeled_df = pd.DataFrame({"Text": unlabeled_texts})
unlabeled_df = kmeans.predict(X_pca_unlabeled)

Best Number of Clusters: 2


In [7]:
# assingn the predicted class to the cases without the keyword
labe_data_df=df.query("case_status=='dismiss appeal' | case_status=='allow appeal'")
unlabe_data_df=df.query("case_status=='no'")
unlabe_data_df["case_status"]= pd.Series(unlabeled_df).map({1:"dismiss appeal", 0:"allow appeal"}).values

In [8]:
# join the unlabel and label dataframe together
df=pd.concat([labe_data_df, unlabe_data_df], axis=0)

In [9]:
# save the dataframe into excel
df.to_excel("court_case.xlsx", index=False)